In [13]:
# Import the needed libraries
import pandas as pd
import numpy as np
import csv
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk

In [3]:
# Create a dataframe to hold our various reviews
df = pd.read_csv('Train.csv')
stop_words_df = pd.read_csv("/Users/0x23/Desktop/*/Projects/Swahili/Swahili_Stopwords.csv")
stop_words = stop_words_df['StopWords'].tolist()

In [5]:
# We creat a list to hold items i.e. sentences from our imported text
text_lines = list()
lines = df['content'].values.tolist()

In [6]:
# We then use a loop to go through the dataframe and preprosses text then add these to the list created earlier
for line in lines:
    tokens = word_tokenize(line)
 
    table = str.maketrans('', '', string.punctuation) # remove punctuation
    stripped = [w.translate(table) for w in tokens] 
    words = [word for word in stripped if word.isalpha()] # remove remaining non alphabetic tokens
    words = [w for w in words if not w in stop_words]
    text_lines.append(words)

In [8]:
# We can then import the Gensim library and use the Word2Vec model to learn the vector represetaions 
import gensim

# Create a vocabulary to hold all words in our corpus. Our vectors will however be limited to this vocab and issue may
# arise when the presented with a new word that wasnt in the corpus.
vocabulary = gensim.models.Word2Vec(sentences=text_lines, size=200, window=5, workers=4, min_count=1)
words = list(vocabulary.wv.vocab)

# Just to see how mayn words we actually have.
print('Vocabulary size:  %d'  % len(words))


# Finally we can export the learnt vectors for use in other tasks e.g. Text Classification
filename = 'swahili_embeddings.txt'
vocabulary.wv.save_word2vec_format(filename, binary=False)

Vocabulary size:  39522


In [12]:
# We can then test to see what words are simillar to a a given word and by what degree/score
vocabulary.wv.most_similar('mtoto')

[('nia', 0.9998518228530884),
 ('msanii', 0.9998043775558472),
 ('mkewe', 0.9997960329055786),
 ('akisema', 0.9997692108154297),
 ('mwili', 0.99974524974823),
 ('kupewa', 0.9997119903564453),
 ('macho', 0.99969881772995),
 ('mke', 0.9996863007545471),
 ('jina', 0.9996820688247681),
 ('News', 0.999665379524231)]

In [14]:
# Export in Format for Visualization in a 3D Space

In [17]:
from gensim.models import KeyedVectors

# Load gensim word2vec
w2v_path = '/Users/0x23/Desktop/*/Projects/Swahili/Zindi/swahili_embeddings.txt'
w2v = KeyedVectors.load_word2vec_format(w2v_path)

import io

# Export Vector file
out_vecs = io.open('swahili_vecs.tsv', 'w', encoding='utf-8')

# Export Meta data file
out_meta = io.open('swahili_meta.tsv', 'w', encoding='utf-8')

# write files
for index in range(len(w2v.index2word)):
    word = w2v.index2word[index]
    vec = w2v.vectors[index]
    out_meta.write(word + "\n")
    out_vecs.write('\t'.join([str(x) for x in vec]) + "\n")
out_vecs.close()
out_meta.close()